# 1 - Invatare seq2seq cu retele neurale

In aceasta serie vom construi un model de machine learning care va mapa o secventa de cuvinte la alta secventa de cuvinte, ceea ce in limbajul de specialitate se cheama seq2seq, folsind Pytorch si TorchText. Acest model il vom aplica pentru a traduce propozitii din romana in engleza, insa se poate aplica si pentru alte taskuri cum ar fi sumarizarea unor documente (se mapeaza o secventa mare de cuvinte la o secventa mica de cuvinte) sau agenti conversationali(chatbots care mapeaza o intrebare la un raspuns).

In acest document vom incerca sa intelegem conceptele generale in implementarea unui model de seq2seq. Pentru a afla mai multe detalii va recomand sa cititi lucrarea https://arxiv.org/abs/1409.3215

## Introducere

Cel mai folosit model *seq2seq* este cel de *encoder-decoder*. Traditional acestea folosesc *retele neurale recurente* drept *encoder* pentru a reprezenta o propozitie data input ca un vector. Putem viziona acest vector ca o reprezentare abstracta a propozitiei primite. Pornind de la aceasta reprezentare alte *retele neurale recurente* drept *decoder* vor primi ca input acest vector si vor invata sa genereze o propozitie primita drept rezultat.

<img src="images/seq2seq2.png">

Imaginea de mai sus este un exemplu despre cum functioneaza acest model. Propozitia "buna dimineata" este primita ca input de catre *stratul embedding* al encoderului. Acest strat din encoder reprezinta cel mai de jos nivel al retelei recurente si invata o reprezentare vectoriala a tuturor cuvintelor din datasetul pe care antrenam reteaua. Pentru a marca inceputul si finalul unei propozitii vom folosi marcatorii (<sos>) si (<eos>) respectiv. Encoderul va primi cuvintele din propozitie pe rand. La fiecare pas de timp, encoderul va primi ca input reprezentarea vectoriala a cuvantului curent din propozitie (aceasta reprezentare este invatata de stratul de embedding din encoder) si starea curenta invatata de *reteaua recurenta*. Aceasta stare este o abstractizare a propozitiei invatata de retea. O putem numi drept *stare ascunsa*. Reteaua recurenta poate fi vazuta ca o functie ce depinde de reprezentarea vectoriala $e(x_t)$ a cuvintelor din propozitii si *starea ascunsa* $h_{t-1}$ a propozitiei invatate pana la pasul $t$:
    $$h_t = \text{EncoderRNN}(e(x_t), h_{t-1})$$

Pentru a deveni putin mai specifici vom utiliza tipuri speciale de *retele recurente* cum ar fi *Long Short Term Memory* sau *Gated Recurrent Unit* pentru *RNN*.

Definim $X = \{x_1, x_2, ..., x_T\}$, $x_1$ = $<sos>$, $x_2$ = $buna$, $x_3$ = $dimineata$,$...$. Starea ascunsa initiala $h_0$ poate fi initializata cu zero sau invatata ca parametru.

La final dupa ce ultimul cuvant $x_T$=$<eos>$ a fost primit de catre catre *RNN*, starea ascunsa finala $h_T$ va fi o reprezentare vectoriala a intregii propozitii.

Avand aceasta reprezentare putem trece la pasul urmator, in care vom genera propozitia "good morning", $Y = \{y_1, y_2, ..., y_T\}$, $y_1$ = $<sos>$, $y_2$ = $good$, $y_3$ = $morning$,$...$. Din nou, la fiecare pas de timp, *RNN* din decoder primeste ca input reprezentarea vectoriala  $d(y_t)$ a cuvantului curent din propozitie si starea curenta  $s_{t-1}$ invatata de *reteaua recurenta* la pasul *t*. Stareainitiala $s_0$ = $h_t$ este aceeasi cu cea primita de la encoder. Asadar similar ecuatiei encoderului avem:
    $$s_t = \text{DecoderRNN}(d(y_t), s_{t-1})$$

Pentru encoder si decoder vom folosi straturi de embedding diferite, deoarece vocabularele de cuvinte corespunzator fiecarei limbi sunt diferite. In decoder, pornind de la vectorul de context primit de la encoder trebuie sa generam un cuvant, asadar la fiecare pas, peste starea curenta $s_t$ se aplica un model liniar pentru a prezice care credem ca ar fi urmatorul cuvant in propozitie, $\hat{y}_t$.

$$\hat{y}_t = f(s_t)$$

In decoder, cuvintele sunt generate unul dupa altul. Cand decoderul primeste $<sos>$ incepe sa genereze iar cand primeste $<eos>$ stie ca trebuie sa se opreasca. O data ce a generat o propozitie, $\hat{Y} = \{ \hat{y}_1, \hat{y}_2, ..., \hat{y}_T \}$ o compara cu propozitia pe care am dori defapt sa o obtinem, $Y = \{ y_1, y_2, ..., y_T \}$. Calculeaza diferenta dintre cele doua propozitii, iar aceasta difenta este trimisa prin *backpropagation* ca sa faca corecteze parametrii modelului pentru a forta modelul sa genereze ce propozitie vrem. Acest model de antrenare se cheama *teacher forcing*, mai multe detalii in cartea lui Bengio https://www.deeplearningbook.org/contents/rnn.html

## Procesare date

Vom folosi Pytorch si TorchText pentru modele si vom folosi spaCy pentru tokenizare text. Vom folosi modelul de la HuggingFace pentru demo tokenizare.

In [217]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset

import spacy
import tokenizers
import numpy as np

import random
import math
import time

from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [218]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In continuare vom crea un tokenizator, unul cu spacy si unul cu hugging face, si vom vedea care e mai rapid:

In [ ]:
from tokenizers import (ByteLevelBPETokenizer,
                            CharBPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)

tokenizer_en = BertWordPieceTokenizer("bert-base-uncased-vocab.txt", lowercase=True)

tokenizer_ro = BertWordPieceTokenizer("roberto_word_level-vocab.txt",lowercase=True)

output = tokenizer_en.encode("Hello boy How are you")
print(output.tokens)
tokenize_ro("Salut baiatul ce mai faci")

Mai departe vom crea functiile de tokenizator. Vor primi o propozitie ca string si vor returna o lista de cuvinte. De exemplu, pentru decoder este mai ok daca ordinea cuvintelor este inversata deoarece aceasta ar creste acuratetea modelelor

In [220]:
def tokenize_en(text):
    return tokenizer_en.encode(text).tokens[::-1]

def tokenize_ro(text):
    return tokenizer_ro.encode(text).tokens

Clasa *Field* din TorchText proceseaza datele de input si de output.
Setam romana ca fiind SRC (sursa input) si engleza ca TRG(target output)

In [438]:
SRC = Field(tokenize = tokenize_ro,
           init_token = "<sos>",
           eos_token = "<eos>",
           lower = True)

TRG  = Field(tokenize=tokenize_en,
            init_token="<sos>",
            eos_token="<eos>",
            lower=True)

Acum luam dataseturile de antrenare, validare si testare. Acestea pot fi ce vrem noi, de la datasets pt chatboti pana la dataseturi de translatie.

Datasetul pe care il vom folosi este 

file_path = os.path.abspath(os.getcwd()) + "/eng_ro_train.csv"

df = pd.read_csv(file_path, sep="\t")
train_data, valid_data = train_test_split(df, test_size=0.1)
train_data.to_csv("train.csv", index=False)
valid_data.to_csv("valid.csv", index=False)

Am creat un dataset de train si un dataset de validare, fiecare cu 2 coloane (engleza, romana) unde fiecare rand contine o propozitie in engleza si o propozitie in romana

Acum vom tokeniza dataseturile in liste de cuvinte conform *Field* definit mai sus pentru fiecare limba: 

In [452]:
data_fields = [('Engleza', TRG), ('Romana', SRC)]

train, val, test = TabularDataset.splits(path="./", train="train.csv",   validation="valid.csv", test='test.csv', format="csv", fields=data_fields)
print(test)
SRC.build_vocab(train, val)
TRG.build_vocab(train, val)


print(f"Cuvinte unice (ro): {len(SRC.vocab)}")
print(f"Cuvinte unice (en): {len(TRG.vocab)}")

Cuvinte unice (ro): 6877
Cuvinte unice (en): 4945


Ultimul pas consta in crearea iteratorilor. Acestia vor returna o secventa de propozitii pe care o va primi ca input sub forma de tensori mapata la o secventa pe care o va avea drept output tot sub forma de tensori. Acesti tensori cuprind defapt indicii cuvintelor din vocabular.

Singurul lucru de are va trebui sa mai avem grija este ca lungimea propozitiilor sa fie fixa. Acest lucru se realizaza prin completarea cu un cuvant dummy numit [PAD]. 

In [453]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [457]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, val, test), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)

## Constructia modelului Seq2Seq

Aceasta cuprinde defapt trei parti: constructia encoder, constructia decoder si la final constructia model seq2sea care sa le imbine pe amandoaua

### Encoder

in lucrarea de mai sus, encoderul contine 4 straturi de LSTM. Noi vom folosi doar 2 pentru rapiditate.

In [455]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        
        return hidden, cell

### Decoder

Ok, in continuare vom construi decoderul care va avea de asemnea doua straturi de LSTM uri.

In [420]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

### Seq2Seq

Pentru partea finala vom implementa seq2seq ...

In [422]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

### Antrenam modelul Seq2Seq

Acum ca aveam modelul construit putem incepe antrenarea....

In [423]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

Initializam weighturile modelului

In [424]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6877, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(4945, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=4945, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculam numarul de parametrii din model

In [425]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Modelul are {count_parameters(model):,} de parametetrii')

Modelul are 12,919,633 de parametetrii


Definim optimizatorul, ceea ce in cazul nostru va fi ori SGD ori ADAM. Pentru mai multe detalii puteti verifica ...

In [426]:
optimizer = optim.Adam(model.parameters())

Urmeaza sa definim functia de loss. Pentru negative log likelyhood merge foarte bine pe traduceri, in schimb pt generare de text merge destul de prost pentru ca invata cuvintele cu cea mai mare frecventa. Desigur aceasta depinde si de decoder care acum este greedy, iar daca il schimbam cu beam search va fi mai bun. Un exemplu bun ar fi sa schimbam functia de negative log liklyhood cu alta functie de loss. Ma gandeasc la retele recurente bayesiane pe viitor.

In [427]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [428]:
output = torch.randn(128, 4945).float()
target = torch.FloatTensor(128).uniform_(0, 120).long()
print(target.shape)
criterion(output, target)

torch.Size([128])


tensor(8.9865)

Acum trebuie sa definim training loop...

In [429]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Romana
        trg = batch.Engleza
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Facem si o functie de evaluare, desi nu avem test dataset...

In [430]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Romana
            trg = batch.Engleza

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Aceasta este o functie care numara cat timp a trecut pentru o epoca de antrenare.

In [431]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Si in sfarsit putem incepe sa antrenam  modelul

In [432]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 35s
	Train Loss: 4.814 | Train PPL: 123.243
	 Val. Loss: 4.347 |  Val. PPL:  77.273
Epoch: 02 | Time: 1m 34s
	Train Loss: 4.281 | Train PPL:  72.347
	 Val. Loss: 4.353 |  Val. PPL:  77.683
Epoch: 03 | Time: 1m 38s
	Train Loss: 4.182 | Train PPL:  65.487
	 Val. Loss: 4.332 |  Val. PPL:  76.071
Epoch: 04 | Time: 1m 39s
	Train Loss: 4.090 | Train PPL:  59.763
	 Val. Loss: 4.364 |  Val. PPL:  78.574
Epoch: 05 | Time: 1m 37s
	Train Loss: 3.993 | Train PPL:  54.214
	 Val. Loss: 4.382 |  Val. PPL:  79.963
Epoch: 06 | Time: 1m 41s
	Train Loss: 3.917 | Train PPL:  50.233
	 Val. Loss: 4.387 |  Val. PPL:  80.408
Epoch: 07 | Time: 1m 40s
	Train Loss: 3.870 | Train PPL:  47.950
	 Val. Loss: 4.394 |  Val. PPL:  80.997
Epoch: 08 | Time: 1m 43s
	Train Loss: 3.815 | Train PPL:  45.377
	 Val. Loss: 4.420 |  Val. PPL:  83.080
Epoch: 09 | Time: 1m 38s
	Train Loss: 3.743 | Train PPL:  42.243
	 Val. Loss: 4.319 |  Val. PPL:  75.116
Epoch: 10 | Time: 1m 42s
	Train Loss: 3.644 | Train PPL

In [458]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

AttributeError: 'Example' object has no attribute 'Engleza'